# Identifying Vehicles from Video data

The code in this notebook performs the task of **taking a video file as input**, and creating **cropped images of the objects detected in the video**.

These cropped images are created as training data for a Deep Learning model. The images will be further labeled into 3 classes - 
1. 2 or 3 wheelers (motorbikes, rickshaws, etc.)
2. 4 wheelers (cars)
3. 4+ wheelers (buses, trucks, etc.)

The overall workflow of this demonstration is as follows:

- We first define some **global variables** that will be used through the entire demo
- Then, we **define functions** that will be used during execution.
- As soon as you run the **while loop** in the main function, the **first frame of the video is initialised**.
- Using your mouse, you will **draw a line across one side of the road**. This will appear as a yellow line.
- Once you draw the line, the video will start running.
- Then, the video will keep running, and **vehicle images will get cropped** and get saved on a specified path.
- When the **vehicle count reaches a certain threshold**, the video will stop.
- All the cropped images will get saved on your machine.

In [11]:
# Run these if OpenCV doesn't load

import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/cv2/')

First, we import the necessary libraries

In [12]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

Next, we define variables that will be used through the duration of the code

In [13]:
# This variable is created so that we can print necessary values while debugging.
# Creating this variable is good general practice.
SHOW_DEBUG_STEPS = False

# This is a boolean variable which decides if a line has been dragged across or not
drag = 0

# This decides if a point has been selected or not
select_flag = 0

# These two points will be the endpoints of the line that we draw
x1=0
y1=0
x2=1
y2=1
point1 = [x1,y1]
point2 = [x2,y2]

# This is a matrix version of the same two points
crossingLine = np.zeros((2,2),np.int)
crossingLine[0] = point1
crossingLine[1] = point2

# This is a boolean variable which determines if the frame is the first frame or not
blnfFrame = True

# Frame count is initialised (This is not initialised to 1 - we throw away the first frame after we draw a line on it)
frameCnt = 2

# This is a variable that's evoked when we are drawing the line on the frame
callback = False

# The count of vehicles is initialised to 0
vehicleCount = 0

# This is the title of the window where the video will play
src_window='Vehicle Counting - Blob Save'

# This is a variable that keeps track of how many blobs have been cropped
u=0

# Here, we define some colours
SCALAR_BLACK = (0.0,0.0,0.0)
SCALAR_WHITE = (255.0,255.0,255.0)
SCALAR_YELLOW = (0.0,255.0,255.0)
SCALAR_GREEN = (0.0,255.0,0.0)
SCALAR_RED = (0.0,0.0,255.0)
SCALAR_CYAN = (255.0,255.0,0.0)

In the next cell, we define the 'Blob' class. 

This has two functions as a part of it: 

- 'Blob', which defines a blob object
- 'predictNextPosition', which predicts the position of the blob in the next frame. This is useful for keeping track that given a frame and a blob, is it a new blob or a previously detected one.

In [14]:
class Blob:
    currentContour = [[0,0]]
    x=0
    y=0
    w=0
    h=0
    currentBoundingRect = [x,y,w,h] # = cv2.boundingRect(_contour)
    centerPositions = [0,0]
    dblCurrentDiagonalSize = 0.0
    dblCurrentAspectRatio = 0.0
    blnCurrentMatchFoundOrNewBlob = False
    blnStillBeingTracked = False
    intNumOfConsecutiveFramesWithoutAMatch = 0
    predictedNextPosition = np.zeros((1,2),np.int)
    
    # First, let's define the 'Blob' function, which creates a 'Blob', with all the necessary parameters
    
    def Blob(self,_contour):
        self.currentContour = _contour
        self.currentBoundingRect = cv2.boundingRect(_contour)
        currentCenter = [0,0]
        
        # This variable defines the center of the blob
        currentCenter[0] = self.currentBoundingRect[0] + self.currentBoundingRect[2] / 2
        currentCenter[1] = self.currentBoundingRect[1] + self.currentBoundingRect[3] / 2
        
        # This is a list of all the center positions of all the blobs.
        self.centerPositions.append(currentCenter)
        
        # These parameters are used to define constraints on blob parameters. 
        self.dblCurrentDiagonalSize = math.sqrt(math.pow(self.currentBoundingRect[2],2) + math.pow(self.currentBoundingRect[3],2))
        self.dblCurrentAspectRatio = float(self.currentBoundingRect[2]) / float(self.currentBoundingRect[3]) 
        
        
        self.blnStillBeingTracked = True
        self.blnCurrentMatchFoundOrNewBlob = True
        
        self.intNumOfConsecutiveFramesWithoutAMatch = 0
        
    
    # Next, we define a function that predicts the next position of the blob
    # This function takes the position of blobs detected in a frame
    # Then it predicts the position of that same blob in subsequent
        
    def predictNextPosition(self):
        numPositions = int(len(self.centerPositions))
                    
        #self.predictedNextPosition[:]=[] 
        if(numPositions==1):
            self.predictedNextPosition[0][0]=self.centerPositions[-1][0] # or [-1:]
            self.predictedNextPosition[0][1]=self.centerPositions[-1][1]
        elif(numPositions==2):
            deltaX = self.centerPositions[1][0] - self.centerPositions[0][0]
            deltaY = self.centerPositions[1][1] - self.centerPositions[0][1]
            self.predictedNextPosition[0][0]=self.centerPositions[-1][0] + deltaX # or [-1:]
            self.predictedNextPosition[0][1]=self.centerPositions[-1][1] + deltaY
        elif(numPositions==3):
            sumOfXChanges = ((self.centerPositions[2][0] - self.centerPositions[1][0])*2) + ((self.centerPositions[1][0] - self.centerPositions[0][0])*1)
            deltaX = int(round(float(sumOfXChanges/3.0)+0.5))
            sumOfYChanges = ((self.centerPositions[2][1] - self.centerPositions[1][1]) * 2) + ((self.centerPositions[1][1] - self.centerPositions[0][1]) * 1)
            deltaY = int(round(float(sumOfYChanges/3.0)+0.5))
            self.predictedNextPosition[0][0]=self.centerPositions[-1][0] + deltaX # or [-1:]
            self.predictedNextPosition[0][1]=self.centerPositions[-1][1] + deltaY
        elif(numPositions==4):
            sumOfXChanges = ((self.centerPositions[3][0] - self.centerPositions[2][0]) * 3) + ((self.centerPositions[2][0] - self.centerPositions[1][0]) * 2) + ((self.centerPositions[1][0] - self.centerPositions[0][0]) * 1)
            deltaX = int(round(float(sumOfXChanges/6.0)+0.5))
            sumOfYChanges = ((self.centerPositions[3][1] - self.centerPositions[2][1]) * 3) + ((self.centerPositions[2][1] - self.centerPositions[1][1]) * 2) + ((self.centerPositions[1][1] - self.centerPositions[0][1]) * 1)
            deltaY = int(round(float(sumOfYChanges/6.0)+0.5))
            self.predictedNextPosition[0][0]=self.centerPositions[-1][0] + deltaX # or [-1:]
            self.predictedNextPosition[0][1]=self.centerPositions[-1][1] + deltaY
        elif(numPositions>=5):
            sumOfXChanges = ((self.centerPositions[numPositions-1][0] - self.centerPositions[numPositions-2][0]) * 4) + ((self.centerPositions[numPositions-2][0] - self.centerPositions[numPositions-3][0]) * 3) + ((self.centerPositions[numPositions-3][0] - self.centerPositions[numPositions-4][0]) * 2) + ((self.centerPositions[numPositions-4][0] - self.centerPositions[numPositions-5][0]) * 1)
            deltaX = int(round(float(sumOfXChanges/10.0)+0.5))
            sumOfYChanges = ((self.centerPositions[numPositions-1][1] - self.centerPositions[numPositions-2][1]) * 4) + ((self.centerPositions[numPositions-2][1] - self.centerPositions[numPositions-3][1]) * 3) + ((self.centerPositions[numPositions-3][1] - self.centerPositions[numPositions-4][1]) * 2) + ((self.centerPositions[numPositions-4][1] - self.centerPositions[numPositions-5][1]) * 1)
            deltaY = int(round(float(sumOfYChanges/10.0)+0.5))
            self.predictedNextPosition[0][0]=self.centerPositions[-1][0] + deltaX # or [-1:]
            self.predictedNextPosition[0][1]=self.centerPositions[-1][1] + deltaY
        else:
            print ("Shouldn't come here")

   

Here, we define all the functions that are used in this demo. If you individually read them, they will be hard to understand.

The recommended way to read this notebook is as follows:
1. Scroll down first to the **main function**.
2. **Start reading** through the main function.
3. Whenever you **encounter a new function, find the function name** (do a Ctrl-F (Cmd-F for Mac) with that function name
4. This will help you **locate the function's definition**. 
5. Then **read through that definition** and understand how the function works

In [15]:

blob = Blob()
blobs = [blob]


# This function closes all the windows that are open with either video or images in them

def closeAll():
    cap.release()
    cv2.destroyAllWindows()
    

# This functions checks if the video is corrupted
    
def retn(ret):
    if not ret == True:
        print("Error reading frame")
        closeAll()

# This functions checks if the first frame is missing        
        
def frm(frame):
    if fFrame is None:
        print("Error reading frame")
        closeAll()
        
# This function is used to draw the yellow line on the first frame.
# This yellow line acts as the boundary - if vehicles cross this line, they are counted as a blob
        
def drawMyLine(frame):
    global point1
    global point2
    global SCALAR_YELLOW
    cv2.line(frame,(point1[0],point1[1]),(point2[0],point2[1]),SCALAR_YELLOW,2,8)
    

    
# This funtion checks if a vehicle is to the left of the line that we have drawn
    
def isLeftOfLineAB(a,b,c):
    return ( (b[0]-a[0])*(c[1]-a[1]) - (b[1]-a[1])*(c[0]-a[0]) ) > 0

# This mouse performs the actual drawing of the line

def myMouseHandler(event,x,y,flags,param): # Click left button to start RoI selection
    global point1
    global point2
    global drag
    global select_flag
    global callback
    if (event == cv2.EVENT_LBUTTONDOWN and not(drag) and not(select_flag)):
        point1 = [x,y]
        drag = 1
    
    if (event == cv2.EVENT_MOUSEMOVE and drag and not(select_flag)): # Drag mouse to select RoI
        img1 = fFrame.copy()
        print ('img1 height' + str(img1.shape[0]))
        print ('img1 width' + str(img1.shape[1]))
        point2 = [x,y]
        drawMyLine(fFrame)
        cv2.imshow(src_window,img1) # why img1?
        
    if(event == cv2.EVENT_LBUTTONUP and drag and not(select_flag)): # Complete selection
        img2 = fFrame.copy()
        print ('img2 height' + str(img2.shape[0]))
        print ('img2 width' + str(img2.shape[1]))
        point2 = [x,y]
        drag = 0
        select_flag = 1
        cv2.imshow(src_window,img2) # why img2?
        callback = 1

# This adds all the blobs in the current frame into the existing list of blobs         
        
def addBlobToExistingBlobs(currentFrameBlob, existingBlobs, intIndex):
    existingBlobs[intIndex].currentContour = currentFrameBlob.currentContour
    existingBlobs[intIndex].currentBoundingRect = currentFrameBlob.currentBoundingRect

    existingBlobs[intIndex].centerPositions.append(currentFrameBlob.centerPositions[-1])

    existingBlobs[intIndex].dblCurrentDiagonalSize = currentFrameBlob.dblCurrentDiagonalSize
    existingBlobs[intIndex].dblCurrentAspectRatio = currentFrameBlob.dblCurrentAspectRatio

    existingBlobs[intIndex].blnStillBeingTracked = True
    existingBlobs[intIndex].blnCurrentMatchFoundOrNewBlob = True

# This is a simple distance calculator    
    
def distanceBetweenPoints(point1, point2):
    if(SHOW_DEBUG_STEPS):
        print ('point1: '+str(type(point1))+'point1: '+str(point1))
        print ('point2: '+str(type(point2))+'point2: '+str(point2))
    intX = abs(point1[0] - point2[0][0])
    intY = abs(point1[1] - point2[0][1])

    return (math.sqrt(math.pow(intX, 2) + math.pow(intY, 2)))

# This function checks that given a blob, has it been spotted in a previous frame before?
        
def matchCurrentFrameBlobsToExistingBlobs(existingBlobs, currentFrameBlobs):
#    for existingBlob in existingBlobs:
#        existingBlobs[existingBlob].blnCurrentMatchFoundOrNewBlob =  False
#        existingBlobs[existingBlob].predictNextPosition()       
        
    for i in range (len(existingBlobs)):
        existingBlobs[i].blnCurrentMatchFoundOrNewBlob =  False
        existingBlobs[i].predictNextPosition()    
        
    for currentFrameBlob in currentFrameBlobs:
        intIndexOfLeastDistance = 0
        dblLeastDistance = 100000.0
        
        for i in range(len(existingBlobs)):
            if existingBlobs[i].blnStillBeingTracked == True:
                if(SHOW_DEBUG_STEPS):
                    print ("existingBlobs[i].predictedNextPosition: " + str(type(existingBlobs[i].predictedNextPosition)))
                    print ("currentFrameBlob.centerPositions[-1]: " + str(type(currentFrameBlob.centerPositions[-1])))
                dblDistance = distanceBetweenPoints(currentFrameBlob.centerPositions[-1], existingBlobs[i].predictedNextPosition.tolist())
                    
                if dblDistance < dblLeastDistance:
                    dblLeastDistance = dblDistance
                    intIndexOfLeastDistance = i
                    
        if dblLeastDistance < currentFrameBlob.dblCurrentDiagonalSize * 0.5:
            addBlobToExistingBlobs(currentFrameBlob, existingBlobs, intIndexOfLeastDistance)
        else:
            addNewBlob(currentFrameBlob, existingBlobs)
    
    for existingBlob in existingBlobs:
        if existingBlob.blnCurrentMatchFoundOrNewBlob == False:
            existingBlob.intNumOfConsecutiveFramesWithoutAMatch = existingBlob.intNumOfConsecutiveFramesWithoutAMatch + 1
        if existingBlob.intNumOfConsecutiveFramesWithoutAMatch >= 5:
            existingBlob.blnStillBeingTracked = False

# This function adds the blob to the list of blobs in the existing frame

def addNewBlob(currentFrameBlob, existingBlobs):

    currentFrameBlob.blnCurrentMatchFoundOrNewBlob = True

    existingBlobs.append(currentFrameBlob)
                
def drawAndShowContours(wd,ht,contours,strImgName):
    global SCALAR_WHITE
    global SHOW_DEBUG_STEPS
    blank_image = np.zeros((ht,wd,3), np.uint8)
    cv2.drawContours(blank_image,contours,-1,SCALAR_WHITE,-1)
    
    if(SHOW_DEBUG_STEPS):
        cv2.imshow(strImgName,blank_image)
    return blank_image
        
def drawAndShowBlobs(wd,ht,blobs,strImgName):
    global SCALAR_WHITE
    global SHOW_DEBUG_STEPS
    blank_image = np.zeros((ht,wd,3), np.uint8)
    
    contours=[]
    for blob in blobs:
        if blob.blnStillBeingTracked == True:
            contours.append(blob.currentContour)
            
    cv2.drawContours(blank_image,contours,-1,SCALAR_WHITE,-1)
    
    if(SHOW_DEBUG_STEPS):
        cv2.imshow(strImgName,blank_image)
    return blank_image    
    
def checkIfBlobsCrossedTheLine(blobs):
    global point1
    global point2
    global vehicleCount
    blnAtLeastOneBlobCrossedTheLine = False
    
    for blob in blobs:
        if (blob.blnStillBeingTracked == True and len(blob.centerPositions) >= 2):
            prevFrmIdx = int(len(blob.centerPositions)) - 2
            currFrmIdx = int(len(blob.centerPositions)) - 1
            
            if(SHOW_DEBUG_STEPS):
                print ('prevFrmIdx: '+str(prevFrmIdx))
                print ('currFrmIdx: '+str(currFrmIdx))
                print ('blob.centerPositions: '+str(blob.centerPositions[prevFrmIdx][1]))
            c=[1,2]
            d=[1,2]
            #d[0]=blob.centerPositions[prevFrmIdx][0]
            #d[1]=blob.centerPositions[prevFrmIdx][1]
            d=blob.centerPositions[prevFrmIdx]
            #c[0]=blob.centerPositions[currFrmIdx][0]
            #c[1]=blob.centerPositions[currFrmIdx][1]
            c=blob.centerPositions[currFrmIdx]
            
            if (isLeftOfLineAB(point1,point2,c) and not(isLeftOfLineAB(point1,point2,d))):
                vehicleCount = vehicleCount + 1
                blnAtLeastOneBlobCrossedTheLine = True
    
    return blnAtLeastOneBlobCrossedTheLine


# This is the function that draws the blob we spot, onto an image
        
def drawBlobInfoOnImage(blobs, imgFrame2Copy, imgorg, frmCnt, fps):
    global SCALAR_RED   
    global u    
    im_name = "_"
    path = "/Users/jaideepkhare/Documents/neural-networks/vehicle-detection/saved_images/"    
    
    im_names = str(u)+'.jpg\n'
    
    for blob in blobs:
        if blob.blnStillBeingTracked == True:
            x,y,w,h = blob.currentBoundingRect
            cv2.rectangle(imgFrame2Copy, (x,y),(x+w,y+h), SCALAR_RED, 2) 
            condi = frmCnt%5
            if (condi == 0):
                img_1boundingbox = imgorg[y:y+h,x:x+w]
                im_name = path+str(u)+".jpg"
                resizedRoI = cv2.resize(img_1boundingbox, (50,50))
                cv2.imwrite(im_name,resizedRoI)
                if not(u==0):
                    im_names = im_names + str(u)+'.jpg\n'
                u = u+1                
                
    for i in range (len(blobs)):
        if blobs[i].blnStillBeingTracked == True:
            x,y,w,h = blobs[i].currentBoundingRect
            cv2.rectangle(imgFrame2Copy, (x,y),(x+w,y+h), SCALAR_RED, 2)
            #if ((frmCnt-2)%(fps+1) == 0):
            img_1boundingbox = imgorg[y:y+h,x:x+w]
            im_name = path+str(u)+".jpg"
            img_1boundingbox = cv2.resize(img_1boundingbox, (50,50))
            cv2.imwrite(im_name,img_1boundingbox)
            if not(u==0):
                im_names = im_names + str(u)+'.jpg\n'
            u = u+1      


From below this, the main function begins.

In [16]:
#main

cap = cv2.VideoCapture('/Users/jaideepkhare/Documents/neural-networks/vehicle-detection/AundhBridge.mp4')
#cap = cv2.VideoCapture('/media/anand/STB_Data/Projects/ext_projects/Upgrad/Univ_bridge_from_pride.mp4')


if not(cap.isOpened()):
    print("Error reading file")

ret, fFrame  = cap.read()
retn(ret)
frm(fFrame)
#fFrame = cv2.resize(fFrame, (0,0), fx=0.5, fy=0.5)

fGray = cv2.cvtColor(fFrame, cv2.COLOR_BGR2GRAY)
cv2.imshow(src_window, fFrame)
          
ret, fFrame1 = cap.read()
ret, fFrame2 = cap.read()
print ('height' + str(fFrame1.shape[0]))
print ('width' + str(fFrame1.shape[1]))
#fFrame1 = cv2.resize(fFrame1, (0,0), fx=0.5, fy=0.5)
#fFrame2 = cv2.resize(fFrame2, (0,0), fx=0.5, fy=0.5)

cv2.setMouseCallback(src_window,myMouseHandler)
chChkEscKey = 0  
k = 27    
blnFirstFrame = True
frameCnt = 2

height720
width1280


This is the while loop. It is kept open as long as the OpenCV video object is kept open.

In [17]:

while(cap.isOpened()):
    if(callback):
        currentFrameBlob = Blob()
        currentFrameBlobs = [currentFrameBlob]
        img1 = fFrame1.copy()
        img2 = fFrame2.copy()
        if(SHOW_DEBUG_STEPS):
            print ('img1 height' + str(img1.shape[0]))
            print ('img1 width' + str(img1.shape[1]))
            print ('img2 height' + str(img2.shape[0]))
            print ('img2 width' + str(img2.shape[1]))
            
        # Convert the images to colour in order to enable fast processing    
        
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        
        # Add some Gaussian Blur
        
        img1 = cv2.GaussianBlur(img1,(5,5),0)
        img2 = cv2.GaussianBlur(img2,(5,5),0)
        
        
        # This imgDiff variable is the difference between consecutive frames, which is equivalent to detecting movement
        
        imgDiff = cv2.absdiff(img1, img2) 
        
        ret,imgThresh = cv2.threshold(imgDiff,30.0,255.0,cv2.THRESH_BINARY)
        ht = np.size(imgThresh,0)
        wd = np.size(imgThresh,1)
        
        if(SHOW_DEBUG_STEPS):
            cv2.imshow('imgThresh', imgThresh) 
        
        
        # Now, we define structuring elements
            
        strucEle3x3 = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
        strucEle5x5 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
        strucEle7x7 = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))        
        strucEle15x15 = cv2.getStructuringElement(cv2.MORPH_RECT,(15,15))
        
        # Next, we perform dilation and erosion on this
        
        for i in range(2):
            imgThresh = cv2.dilate(imgThresh,strucEle5x5,iterations = 2)
            imgThresh = cv2.erode(imgThresh,strucEle5x5,iterations = 1)
        
        imgThreshCopy = imgThresh.copy()
        if(SHOW_DEBUG_STEPS):        
            print ('imgThreshCopy height' + str(imgThreshCopy.shape[0]))
            print ('imgThreshCopy width' + str(imgThreshCopy.shape[1]))
            
            
        # Now, we move on to the contour mapping portion    
        
        im, contours, hierarchy = cv2.findContours(imgThreshCopy,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        im2 = drawAndShowContours(wd,ht,contours,'imgContours')
            
        if(SHOW_DEBUG_STEPS):
            print ('contours.shape: ' + str(len(contours)))
            
            
        # Next, we define hulls.
        # Hulls are contours with the "convexHull" function from cv2
        
        hulls = contours # does it work?
        for i in range(len(contours)):
            hulls[i] = cv2.convexHull(contours[i])
            
        # Then we draw the contours    
        
        im3 = drawAndShowContours(wd,ht,hulls,'imgConvexHulls')
        
        # Next, we move to blobs
        # First, each hull is passed through the Blob function
            
        for hull in hulls:
            # This is an instance of the class Blob()
            possiBlob = Blob()
            # This is the Blob function inside the class Blob()
            possiBlob.Blob(hull) # does it work? yes
            currentBoundingRectArea = possiBlob.currentBoundingRect[2] * possiBlob.currentBoundingRect[3] 
            contourArea = cv2.contourArea(hull)
            # These are the conditions which determine whether a hull can satisfactorily be called a Blob
            if(currentBoundingRectArea > 400 and possiBlob.dblCurrentAspectRatio > 0.2 and possiBlob.dblCurrentAspectRatio < 4.0 and possiBlob.currentBoundingRect[2] > 30 and possiBlob.currentBoundingRect[3] > 30 and possiBlob.dblCurrentDiagonalSize > 60.0 and (contourArea/int(currentBoundingRectArea) > 0.5)):
                currentFrameBlobs.append(possiBlob)
        
        # Now, using the hulls, we draw the blob objects.
        
        im4 = drawAndShowBlobs(wd,ht,currentFrameBlobs,'imgCurrentFrameBlobs')
            
        if blnFirstFrame ==  True:
            for currFrameBlob in currentFrameBlobs:
                blobs.append(currFrameBlob)
        else:
            matchCurrentFrameBlobsToExistingBlobs(blobs, currentFrameBlobs)
        
        im5 = drawAndShowBlobs(wd,ht, blobs, "imgBlobs")
        
        img2 = fFrame2.copy()
        if(SHOW_DEBUG_STEPS):
            print ('img2 height' + str(img2.shape[0]))
            print ('img2 width' + str(img2.shape[1]))
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        drawBlobInfoOnImage(blobs, img2, fFrame2, frameCnt, fps)
        
        # Now, we assign the Boolean variable to whether the Blob has crossed the line we drew or not
        
        #blnAtLeastOneBlobCrossedTheLine = checkIfBlobsCrossedTheLine(blobs, /*intHorizontalLinePosition,*/ vehicleCount)
        blnAtLeastOneBlobCrossedTheLine = checkIfBlobsCrossedTheLine(blobs)
        
        # 
        
        if(SHOW_DEBUG_STEPS):
            print ('type: ')
            print (type(crossingLine[0]))
        if blnAtLeastOneBlobCrossedTheLine == True:
#            cv2.line(img2, (crossingLine[0]), (crossingLine[1]),(0,255,0),2)
            cv2.line(img2, (point1[0],point1[1]),(point2[0],point2[1]),SCALAR_GREEN,2)
        else:
            cv2.line(img2, (point1[0],point1[1]),(point2[0],point2[1]),SCALAR_RED,2)
            
        # drawvehicleCountOnImage(img2)
        drawMyLine(img2) # is it necessary?
        
        cv2.imshow(src_window,img2)
        
        currentFrameBlobs[:]=[] # clear list
        
        fFrame1 = fFrame2.copy()
        ret, fFrame2 = cap.read()
        retn(ret)
        frm(fFrame2)
        #fFrame2 = cv2.resize(fFrame2, (0,0), fx=0.5, fy=0.5) 
        if(SHOW_DEBUG_STEPS):
            print ('img1 height' + str(fFrame1.shape[0]))
            print ('img1 width' + str(fFrame1.shape[1]))
            print ('img2 height' + str(fFrame2.shape[0]))
            print ('img2 width' + str(fFrame2.shape[1]))        
        
        blnFirstFrame = False
        frameCnt = frameCnt + 1
    
    k = cv2.waitKey(1) 
    if k == 27 or k == ord('q') or vehicleCount > 20:
        closeAll()
        break
        
closeAll()

img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 height720
img1 width1280
img1 heigh